# DATA2001 Group Assingment 

In [1]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

## Task 1

Load all the libraries needed.

In [2]:
import pandas as pd 
import numpy as np
import plotly.express as px 
import requests 
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json
import os
import plotly.io as pio
pio.renderers.default = 'iframe'

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

Check that connection to PGAdmin server is successful.

In [3]:
db, conn = pgconnect(credentials)

Connected successfully.


Use the following helper function which produces all output as a Pandas dataframe by default (and handles other small things like incorporating the `.text()` function):

In [4]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

Next, we define the schema and populate the table for each of the datasets needed. In the process some data cleaning has to be done to deal with NULL placeholders in certain datasets. We have a combination of spatial data and usual normalised data to load into PostgreSQL server. Also take care of indexing for each dataset. 

First, we do this for income data. Data cleaning needs to done for this dataset so 'np' place holder for NULL needs to be converted to 0 for numerical columns

In [5]:
incomedata = pd.read_csv('Income.csv')
incomedata.columns = map(str.lower, incomedata.columns)

incomedata = incomedata.replace('np', pd.NA)


numeric_cols = ['earners', 'median_income', 'mean_income']
for col in numeric_cols:
    incomedata[col] = pd.to_numeric(incomedata[col], errors='coerce')

incomedata[numeric_cols] = incomedata[numeric_cols].fillna(0)

incomedata.head()

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467.0,51,46640.0,68904.0
1,101021008,Karabar,5103.0,42,65564.0,69672.0
2,101021009,Queanbeyan,7028.0,39,63528.0,69174.0
3,101021010,Queanbeyan - East,3398.0,39,66148.0,74162.0
4,101021012,Queanbeyan West - Jerrabomberra,8422.0,44,78630.0,91981.0


In [6]:
conn.execute(text("""
DROP TABLE IF EXISTS income;
CREATE TABLE income(
    sa2_code21 INT PRIMARY KEY,
    sa2_name VARCHAR(100),
    earners INT,
    median_income NUMERIC,
    mean_income NUMERIC
);
    CREATE INDEX income_median_income_ind ON income(median_income)
"""))


incomedata = incomedata[['sa2_code21', 'sa2_name', 'earners', 'median_income', 'mean_income']]
incomedata.to_sql("income", con=conn, if_exists='append', index=False)
query(conn, "select * from income")

,sa2_code21,sa2_name,earners,median_income,mean_income
0,101021007,Braidwood,2467,46640.0,68904.0
1,101021008,Karabar,5103,65564.0,69672.0
2,101021009,Queanbeyan,7028,63528.0,69174.0
3,101021010,Queanbeyan - East,3398,66148.0,74162.0
4,101021012,Queanbeyan West - Jerrabomberra,8422,78630.0,91981.0
...,...,...,...,...,...
637,128021537,Royal National Park,14,36980.0,47584.0
638,128021538,Sutherland - Kirrawee,13895,64940.0,74867.0
639,128021607,Engadine,10239,63695.0,72995.0
640,128021608,Loftus - Yarrawarrah,4424,63087.0,76440.0


Next, for businesses data. In this case we need to make a composite primary key since there are several industries in the same SA2 area so SA2 code cannot be unique in each row. 

In [7]:
businessdata = pd.read_csv('Businesses.csv')
businessdata.columns = map(str.lower, businessdata.columns)

businessdata.columns

Index(['industry_code', 'industry_name', 'sa2_code', 'sa2_name',
       '0_to_50k_businesses', '50k_to_200k_businesses',
       '200k_to_2m_businesses', '2m_to_5m_businesses', '5m_to_10m_businesses',
       '10m_or_more_businesses', 'total_businesses'],
      dtype='object')

In [8]:
conn.execute(text("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses(
    sa2_code INT,
    sa2_name VARCHAR(100),
    industry_code CHAR(1),
    industry_name VARCHAR(100),
    total_businesses INT,
    PRIMARY KEY (sa2_code, industry_code)
);
    CREATE INDEX businesses_sa2_code_ind ON businesses(sa2_code)         
"""))

businessdata = businessdata[['sa2_code', 'sa2_name', 'industry_code', 'industry_name', 'total_businesses']]
businessdata.to_sql("businesses", con=conn, if_exists= 'append', index = False)
query(conn, "select * from businesses")

,sa2_code,sa2_name,industry_code,industry_name,total_businesses
0,101021007,Braidwood,A,"Agriculture, Forestry and Fishing",296
1,101021008,Karabar,A,"Agriculture, Forestry and Fishing",9
2,101021009,Queanbeyan,A,"Agriculture, Forestry and Fishing",15
3,101021010,Queanbeyan - East,A,"Agriculture, Forestry and Fishing",3
4,101021012,Queanbeyan West - Jerrabomberra,A,"Agriculture, Forestry and Fishing",16
...,...,...,...,...,...
12212,128021538,Sutherland - Kirrawee,S,Other Services,152
12213,128021607,Engadine,S,Other Services,87
12214,128021608,Loftus - Yarrawarrah,S,Other Services,22
12215,128021609,Woronora Heights,S,Other Services,9


Next, for population data. In task 3 the schools metric accounts for demographic at the age range of 0-19 so we will create such a column for convenience. 

In [9]:
popdata = pd.read_csv('Population.csv')
popdata.columns = map(str.lower, popdata.columns)

popdata['young_people'] = (
    popdata['0-4_people'] +
    popdata['5-9_people'] +
    popdata['10-14_people'] +
    popdata['15-19_people']
)

popdata.head()

,sa2_code,sa2_name,0-4_people,5-9_people,10-14_people,15-19_people,20-24_people,25-29_people,30-34_people,35-39_people,...,50-54_people,55-59_people,60-64_people,65-69_people,70-74_people,75-79_people,80-84_people,85-and-over_people,total_people,young_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,602,570,520,464,369,226,142,70,7530,2121
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,794,895,863,925,603,331,264,11052,2471
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,436,422,397,327,264,190,100,75,4748,961
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,882,901,930,917,1065,976,773,1028,14803,3205
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1241,1377,1285,1166,949,664,476,537,21346,4364


In [10]:
conn.execute(text("""
DROP TABLE IF EXISTS population;
CREATE TABLE population(
    sa2_code INT PRIMARY KEY,
    sa2_name VARCHAR(100),
    total_people INT, 
    young_people INT
);
"""))


popdata = popdata[['sa2_code', 'sa2_name', 'total_people', 'young_people']]
popdata.to_sql("population", con=conn, if_exists= 'append', index = False)
query(conn, "select * from population")

,sa2_code,sa2_name,total_people,young_people
0,102011028,Avoca Beach - Copacabana,7530,2121
1,102011029,Box Head - MacMasters Beach,11052,2471
2,102011030,Calga - Kulnura,4748,961
3,102011031,Erina - Green Point,14803,3205
4,102011032,Gosford - Springfield,21346,4364
...,...,...,...,...
368,128021537,Royal National Park,45,20
369,128021538,Sutherland - Kirrawee,23369,5078
370,128021607,Engadine,17379,5118
371,128021608,Loftus - Yarrawarrah,7354,2073


After done with normal data, we will ingest spatial data which are SA2 Regions, Stops and Schools (In this order).

In [11]:
sa2_regions = gpd.read_file("SA2_2021_AUST_GDA2020.shp")
sa2_regions = sa2_regions[sa2_regions['GCC_NAME21'] == 'Greater Sydney']
sa2_regions.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.41373 -33.46558, 151.41362 -33.4..."
29,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.37484 -33.50052, 151.37507 -33.5..."
30,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((151.20449 -33.5328, 151.20448 ..."
31,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.37194 -33.43698, 151.37288 -33.4..."
32,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((151.32349 -33.42779, 151.32342 -33.4..."


The functions we'll be leveraging for geographical operations rely on PostGIS (the spatial extension to PostgreSQL) being installed on the database - the below query confirms it is correctly configured.


In [12]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


Next we will transform the polygons whilst conducting the WKT conversion. In this case we use GDA2020 so srid is 7844 (seen from the prj file of the dataset). However, for the consistency between spatial tables when used for Task 3, we will use 4283 for all datasets. 

In [13]:
srid = 4283
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

sa2_regions_og = sa2_regions.copy()  # creating a copy of the original for later
sa2_regions['geom'] = sa2_regions['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sa2_regions = sa2_regions.drop(columns="geometry")  # deleting the old copy

In [14]:
sa2_regions.columns = map(str.lower, sa2_regions.columns)
sa2_regions.head()

,sa2_code21,sa2_name21,chg_flag21,chg_lbl21,sa3_code21,sa3_name21,sa4_code21,sa4_name21,gcc_code21,gcc_name21,ste_code21,ste_name21,aus_code21,aus_name21,areasqkm21,loci_uri21,geom
28,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.413733024921 -33.465580583...
29,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37484081570685 -33.5005199...
30,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.20449037540152 -33.5328022...
31,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.37193611462118 -33.4369790...
32,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,MULTIPOLYGON (((151.32348639265098 -33.4277852...


Now we can ingest the sa2 region dataset into PostgreSQL server. Take care of spatial index.

In [15]:
conn.execute(text("""DROP TABLE IF EXISTS sa2_regions;
CREATE TABLE sa2_regions (
    sa2_code21 INT PRIMARY KEY,
    sa2_name21 VARCHAR(100),
    sa4_code21 INT,
    sa4_name21 VARCHAR(100),
    areasqkm21 NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);   
    CREATE INDEX sa2_regions_geom_ind ON sa2_regions USING GIST (geom)"""
))

sa2_regions = sa2_regions[['sa2_code21', 'sa2_name21', 'sa4_code21', 'sa4_name21', 'areasqkm21', 'geom']]
sa2_regions.to_sql("sa2_regions", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from sa2_regions limit 10")

,sa2_code21,sa2_name21,sa4_code21,sa4_name21,areasqkm21,geom
0,102011028,Avoca Beach - Copacabana,102,Central Coast,6.4376,0106000020BB100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,102,Central Coast,32.0802,0106000020BB1000000100000001030000000100000010...
2,102011030,Calga - Kulnura,102,Central Coast,767.9512,0106000020BB1000000200000001030000000100000085...
3,102011031,Erina - Green Point,102,Central Coast,33.7934,0106000020BB1000000100000001030000000100000041...
4,102011032,Gosford - Springfield,102,Central Coast,16.9123,0106000020BB100000010000000103000000010000007E...
5,102011033,Kariong,102,Central Coast,8.3063,0106000020BB100000010000000103000000010000000F...
6,102011034,Kincumber - Picketts Valley,102,Central Coast,11.7169,0106000020BB1000000100000001030000000100000029...
7,102011035,Narara,102,Central Coast,7.7021,0106000020BB1000000100000001030000000100000009...
8,115011555,Castle Hill - North,115,Sydney - Baulkham Hills and Hawkesbury,6.8149,0106000020BB10000001000000010300000001000000FA...
9,119031667,Penshurst,119,Sydney - Inner South West,1.6722,0106000020BB10000001000000010300000001000000A7...


Next is the public transport stops data. 

In [16]:
stops = pd.read_csv('Stops.txt')
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code
0,200039,200039.0,"Central Station, Eddy Av, Stand A",-33.882206,151.206665,NaN,200060,0,NaN
1,200054,200054.0,"Central Station, Eddy Av, Stand D",-33.882042,151.206991,NaN,200060,0,NaN
2,200060,NaN,Central Station,-33.884084,151.206292,1.0,NaN,0,NaN
3,201510,NaN,Redfern Station,-33.891690,151.198866,1.0,NaN,0,NaN
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",-33.893329,151.198882,NaN,201510,0,NaN


In [17]:
stops['geom'] = gpd.points_from_xy(stops.stop_lat, stops.stop_lon)  # creating the geometry column
stops = stops.drop(columns=['stop_lat', 'stop_lon'])  # removing the old latitude/longitude fields
stops.head()

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,NaN,POINT (-33.882 151.207)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,NaN,POINT (-33.882 151.207)
2,200060,NaN,Central Station,1.0,NaN,0,NaN,POINT (-33.884 151.206)
3,201510,NaN,Redfern Station,1.0,NaN,0,NaN,POINT (-33.892 151.199)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,NaN,POINT (-33.893 151.199)


In [18]:
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
stops

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,NaN,POINT (-33.8822064874687 151.20666465471)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,NaN,POINT (-33.8820421431408 151.20699145565)
2,200060,NaN,Central Station,1.0,NaN,0,NaN,POINT (-33.8840842535493 151.206292455081)
3,201510,NaN,Redfern Station,1.0,NaN,0,NaN,POINT (-33.8916900512711 151.198866071817)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,NaN,POINT (-33.8933293130144 151.198881722942)
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",NaN,21271,1,B,POINT (-33.8220164586429 151.07879697831)
114714,2137185,2137185.0,"Cabarita Wharf, Side A",NaN,21371,1,1A,POINT (-33.8406690716775 151.116926480557)
114715,2137186,2137186.0,"Cabarita Wharf, Side B",NaN,21371,1,1B,POINT (-33.8407691073139 151.116898892402)
114716,21501,21501.0,Parramatta Wharf,NaN,2150112,1,NaN,POINT (-33.8139042429414 151.010576673346)


In [19]:
conn.execute(text("""DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id VARCHAR(20) PRIMARY KEY,
    stop_name VARCHAR(100),
    wheelchair_boarding INT,
    geom GEOMETRY(POINT,4283)
);
    CREATE INDEX stops_geom_ind ON stops USING GIST (geom)"""
))

stops = stops[['stop_id', 'stop_name', 'wheelchair_boarding', 'geom']]
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
query(conn, "select * from stops limit 10")

,stop_id,stop_name,wheelchair_boarding,geom
0,200039,"Central Station, Eddy Av, Stand A",0,0101000020BB100000A1FF6524ECF040C0FFA631FF9CE6...
1,200054,"Central Station, Eddy Av, Stand D",0,0101000020BB100000E33DC7C1E6F040C02F928BAC9FE6...
2,200060,Central Station,0,0101000020BB1000008FF33DAC29F140C0817FA2F299E6...
3,201510,Redfern Station,0,0101000020BB10000060304CE622F240C09E57611C5DE6...
4,201646,"Redfern Station, Gibbons St, Stand B",0,0101000020BB1000003DFA6B9D58F240C0DBF9333D5DE6...
5,204230,"St Peters Station, King St",0,0101000020BB10000076DF921A02F440C0B31F3BB6CBE5...
6,204311,King St Opp St Peters Station,0,0101000020BB100000627AB7A805F440C0AFE292CACDE5...
7,204313,Erskineville Rd At Charles St,0,0101000020BB1000004F71D1CD29F340C04D466749E9E5...
8,204320,Erskineville Rd At Prospect St,0,0101000020BB1000007FD9B1E328F340C0AC12B4FDE2E5...
9,204410,St Peters Station,0,0101000020BB100000FECD638521F440C06EC9C4A4C8E5...


Next is the schools datasets. Since datasets are divided into 3 types of school, we will generate 3 tables.

In [20]:
primary_sch = gpd.read_file("catchments_primary.shp")

primary_sch.columns = map(str.lower, primary_sch.columns)
primary_sch.head()

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
3,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.7296, 150.92602 -33.72..."
4,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.15379 -33.75586, 151.15404 -33.7..."


In [21]:
primary_sch_og = primary_sch.copy()  # creating a copy of the original for later
primary_sch['geom'] = primary_sch['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
primary_sch = primary_sch.drop(columns="geometry")  # deleting the old copy
primary_sch.head()

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geom
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.93563850416004 -33.7161211...
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.1833640465581 -33.74748398...
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.0451821055135 -33.77303212...
3,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.92567327976582 -33.7296030...
4,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.1537883781186 -33.75586174...


In [22]:
conn.execute(text("""DROP TABLE IF EXISTS primary_sch;
CREATE TABLE primary_sch (
    use_id INT PRIMARY KEY,
    catch_type VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);   
    CREATE INDEX primary_sch_geom_ind ON primary_sch USING GIST (geom)"""
))

primary_sch = primary_sch[['use_id', 'catch_type', 'geom']]
primary_sch.to_sql("primary_sch", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from primary_sch")

,use_id,catch_type,geom
0,2838,PRIMARY,0106000020BB1000000100000001030000000100000078...
1,2404,PRIMARY,0106000020BB10000001000000010300000001000000BE...
2,4393,PRIMARY,0106000020BB1000000100000001030000000100000065...
3,4405,PRIMARY,0106000020BB1000000100000001030000000100000048...
4,4615,PRIMARY,0106000020BB1000000100000001030000000100000056...
...,...,...,...
1657,2409,PRIMARY,0106000020BB10000001000000010300000001000000C9...
1658,4383,PRIMARY,0106000020BB10000001000000010300000001000000B9...
1659,3275,PRIMARY,0106000020BB10000001000000010300000001000000C0...
1660,2239,PRIMARY,0106000020BB10000001000000010300000001000000F6...


In [23]:
sec_sch = gpd.read_file("catchments_secondary.shp")

sec_sch.columns = map(str.lower, sec_sch.columns)
sec_sch.head()

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.67182 -35.31444, 146.6893 -35.31..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.81448 -35.78341, 146.8125 -35.79..."
3,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"MULTIPOLYGON (((150.686 -33.74031, 150.68631 -..."
4,8559,HIGH_COED,Jamison HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((150.69513 -33.75627, 150.68936 -33.7..."


In [24]:
sec_sch_og = sec_sch.copy()  # creating a copy of the original for later
sec_sch['geom'] = sec_sch['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sec_sch = sec_sch.drop(columns="geometry")  # deleting the old copy
sec_sch.head()

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geom
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,MULTIPOLYGON (((146.67182402032344 -35.3144375...
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,MULTIPOLYGON (((147.08733806259178 -35.8627146...
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,MULTIPOLYGON (((146.81447829547324 -35.7834062...
3,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,MULTIPOLYGON (((150.68599834118749 -33.7403060...
4,8559,HIGH_COED,Jamison HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,MULTIPOLYGON (((150.69513440644116 -33.7562688...


In [25]:
conn.execute(text("""DROP TABLE IF EXISTS sec_sch;
CREATE TABLE sec_sch (
    use_id INT PRIMARY KEY,
    catch_type VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);   
    CREATE INDEX sec_sch_geom_ind ON sec_sch USING GIST (geom)"""
))

sec_sch = sec_sch[['use_id', 'catch_type', 'geom']]
sec_sch.to_sql("sec_sch", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from sec_sch")

,use_id,catch_type,geom
0,8503,HIGH_COED,0106000020BB100000010000000103000000010000006D...
1,8266,HIGH_COED,0106000020BB1000000100000001030000000100000071...
2,8559,HIGH_COED,0106000020BB100000010000000103000000020000002F...
3,8502,HIGH_COED,0106000020BB100000010000000103000000010000003B...
4,8505,HIGH_COED,0106000020BB100000010000000103000000010000003F...
...,...,...,...
431,8213,HIGH_BOYS,0106000020BB1000000100000001030000000100000040...
432,8108,HIGH_COED,0106000020BB10000001000000010300000001000000AD...
433,3235,CENTRAL_HIGH,0106000020BB100000010000000103000000010000003E...
434,1115,CENTRAL_HIGH,0106000020BB100000010000000103000000010000005B...


In [26]:
f_sch = gpd.read_file("catchments_future.shp")

f_sch.columns = map(str.lower, f_sch.columns)
f_sch_og = f_sch.copy()  # creating a copy of the original for later
f_sch['geom'] = f_sch['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
f_sch = f_sch.drop(columns="geometry")  # deleting the old copy
f_sch.head()

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,geom
0,8416,HIGH_COED,Ku-ring-gai HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.19848917708944 -33.5398987...
1,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.27151530428182 -33.9140183...
2,8539,HIGH_COED,SSC Blackwattle Bay,20220609,0,0,0,0,0,0,0,0,0,0,0,2024,2024,MULTIPOLYGON (((151.15292370935092 -33.8393921...
3,8400,HIGH_COED,St Ives HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.17793729938725 -33.6982001...
4,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,MULTIPOLYGON (((151.28072275958445 -33.8328728...


In [27]:
conn.execute(text("""DROP TABLE IF EXISTS f_sch;
CREATE TABLE f_sch (
    use_id INT PRIMARY KEY,
    catch_type VARCHAR(50),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);   
    CREATE INDEX f_sch_geom_ind ON f_sch USING GIST (geom)"""
))

f_sch = f_sch[['use_id', 'catch_type', 'geom']]
f_sch.to_sql("f_sch", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from f_sch")

,use_id,catch_type,geom
0,8416,HIGH_COED,0106000020BB1000000100000001030000000100000090...
1,8161,HIGH_BOYS,0106000020BB100000010000000103000000010000006F...
2,8539,HIGH_COED,0106000020BB10000001000000010300000001000000E3...
3,8286,HIGH_COED,0106000020BB1000000100000001030000000100000099...
4,4524,PRIMARY,0106000020BB1000000100000001030000000100000060...
5,8554,HIGH_COED,0106000020BB10000001000000010300000001000000FE...
6,8400,HIGH_COED,0106000020BB1000000100000001030000000100000060...
7,8555,HIGH_COED,0106000020BB100000010000000103000000010000000C...
8,8135,HIGH_COED,0106000020BB100000010000000103000000010000001F...
9,8556,CENTRAL_HIGH,0106000020BB1000000100000001030000000100000028...


# Task 2 

We will query the NSW POI API

In [28]:
response = requests.get("https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer")
response

<Response [200]>

Develop a function that returns all points of interests from the API within a specified bounding box of coordinates.

In [29]:
def pois_within_bbox(minx, miny, maxx, maxy, filters={}):
    baseURL = 'https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query'

    params = {
        'geometry': f'{minx},{miny},{maxx},{maxy}',
        'geometryType': 'esriGeometryEnvelope',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': '*',
        'returnGeometry': 'true',
        'f': 'json'
    }
    response = requests.get(baseURL, params)
    return json.loads(response.text)['features']

For member 530156989, the chosen SA4 is Blacktown.

In [30]:
sa2_regions_blacktown = sa2_regions_og[sa2_regions_og['SA4_NAME21'] == 'Sydney - Blacktown']
sa2_regions_blacktown.head()

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
319,116011303,Blacktown (East) - Kings Park,0,No change,11601,Blacktown,116,Sydney - Blacktown,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.9753,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.90493 -33.7601, 150.9049 -33.759..."
320,116011304,Blacktown (North) - Marayong,0,No change,11601,Blacktown,116,Sydney - Blacktown,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.3828,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.87753 -33.74582, 150.87697 -33.7..."
321,116011306,Doonside - Woodcroft,0,No change,11601,Blacktown,116,Sydney - Blacktown,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,9.9106,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.85568 -33.76181, 150.85563 -33.7..."
322,116011307,Lalor Park - Kings Langley,0,No change,11601,Blacktown,116,Sydney - Blacktown,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,11.7998,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.92449 -33.74982, 150.92363 -33.7..."
323,116011560,Blacktown - South,3,Name change,11601,Blacktown,116,Sydney - Blacktown,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.3594,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((150.89966 -33.78617, 150.89971 -33.7..."


Build a loop that cycles through each SA2 region within the selected SA4 region, then runs the function for that region’s bounding box, to find all points of interest within that SA2 region.

In [31]:
import time

blacktown_pois = []

for idx, row in sa2_regions_blacktown.iterrows():
    sa2_code = row['SA2_CODE21']
    bounds = row['geometry'].bounds

    features = pois_within_bbox(*bounds)
    
    for feat in features:
        props = feat['attributes']
        props['SA2_CODE21'] = sa2_code
        blacktown_pois.append(props)
    
    print(f"Fetched {len(features)} POIs for SA2 {sa2_code}")
    time.sleep(1)  

Fetched 196 POIs for SA2 116011303
Fetched 114 POIs for SA2 116011304
Fetched 114 POIs for SA2 116011306
Fetched 217 POIs for SA2 116011307
Fetched 58 POIs for SA2 116011560
Fetched 76 POIs for SA2 116011561
Fetched 191 POIs for SA2 116011626
Fetched 69 POIs for SA2 116011627
Fetched 89 POIs for SA2 116021309
Fetched 9 POIs for SA2 116021562
Fetched 78 POIs for SA2 116021563
Fetched 84 POIs for SA2 116021628
Fetched 62 POIs for SA2 116021629
Fetched 156 POIs for SA2 116021630
Fetched 45 POIs for SA2 116021631
Fetched 101 POIs for SA2 116021632
Fetched 60 POIs for SA2 116021633
Fetched 126 POIs for SA2 116031313
Fetched 51 POIs for SA2 116031314
Fetched 97 POIs for SA2 116031315
Fetched 136 POIs for SA2 116031316
Fetched 185 POIs for SA2 116031317
Fetched 167 POIs for SA2 116031318
Fetched 266 POIs for SA2 116031319


In [32]:
blacktown_poi_df = pd.DataFrame(blacktown_pois)
blacktown_poi_df.columns = map(str.lower, blacktown_poi_df.columns)
blacktown_poi_df.head()

,objectid,topoid,poigroup,poitype,poiname,poilabel,poilabeltype,poialtlabel,poisourcefeatureoid,accesscontrol,startdate,enddate,lastupdate,msoid,centroidid,shapeuuid,changetype,processstate,urbanity,sa2_code21
0,1836,500245990,3,Park,BREWONGLE WALKWAY,BREWONGLE WALKWAY,NAMED,None,61,1,1285588392000,32503680000000,1285588392535,89017,None,0b0df18d-085e-30f0-b593-ad0595c63321,I,None,U,116011303
1,1837,500246012,3,Park,None,Park,GENERIC,None,61,1,1285588392000,32503680000000,1285588392535,93537,None,70934b1e-9583-3460-a0bd-8b2712535cc8,I,None,U,116011303
2,1841,500246067,3,Park,None,Park,GENERIC,None,61,1,1285588392000,32503680000000,1285588392535,85402,None,66e1a476-5450-3b8a-aaab-dbd4107fec0f,I,None,U,116011303
3,1842,500246078,3,Park,None,Park,GENERIC,None,61,1,1285588392000,32503680000000,1285588392535,93109,None,2a3eaa8a-3fb9-3b1c-a730-a460c71dd892,I,None,U,116011303
4,1849,500246167,3,Park,None,Park,GENERIC,None,61,1,1285588392000,32503680000000,1285588392535,87938,None,b3980d91-6ab7-3b3d-bf74-0a793d2263e1,I,None,U,116011303


In [33]:
conn.execute(text("""DROP TABLE IF EXISTS blacktown_pois;
CREATE TABLE blacktown_pois (
    sa2_code21 INT,
    topoid INT,
    poigroup INT,
    poitype VARCHAR(100),
    poiname VARCHAR(100),
    poilabel VARCHAR(100),
    poilabeltype VARCHAR(50),
    PRIMARY KEY (sa2_code21, topoid)
);"""))

blacktown_poi_df = blacktown_poi_df[['sa2_code21', 'topoid', 'poigroup', 'poitype', 'poiname', 'poilabel', 'poilabeltype']]
blacktown_poi_df.to_sql("blacktown_pois", con=conn, if_exists= 'append', index = False)
query(conn, "select * from blacktown_pois limit 10")

,sa2_code21,topoid,poigroup,poitype,poiname,poilabel,poilabeltype
0,116011303,500245990,3,Park,BREWONGLE WALKWAY,BREWONGLE WALKWAY,NAMED
1,116011303,500246012,3,Park,None,Park,GENERIC
2,116011303,500246067,3,Park,None,Park,GENERIC
3,116011303,500246078,3,Park,None,Park,GENERIC
4,116011303,500246167,3,Park,None,Park,GENERIC
5,116011303,500246189,3,Park,None,Park,GENERIC
6,116011303,500246200,3,Park,None,Park,GENERIC
7,116011303,500246212,3,Park,None,Park,GENERIC
8,116011303,500246223,3,Park,None,Park,GENERIC
9,116011303,500246256,3,Park,JOSEPH FRANK PARK,JOSEPH FRANK PARK,NAMED


# Task 3

We will build a query pipeline by starting off with the metric components and then standardizing them as z-score, finally combining them into a sigmoid score. 

We check for each of our zone if there exist any sa2 areas with a population below 100. 

In [34]:
sql = """
select * 
from population 
where total_people >= 100;
"""

query(conn, sql)  

,sa2_code,sa2_name,total_people,young_people
0,102011028,Avoca Beach - Copacabana,7530,2121
1,102011029,Box Head - MacMasters Beach,11052,2471
2,102011030,Calga - Kulnura,4748,961
3,102011031,Erina - Green Point,14803,3205
4,102011032,Gosford - Springfield,21346,4364
...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,22151,6212
356,128021538,Sutherland - Kirrawee,23369,5078
357,128021607,Engadine,17379,5118
358,128021608,Loftus - Yarrawarrah,7354,2073


Businesses per 1000 people, in selected industries.
For selected industries, we picked finacial service industry, whole sale, retail sale, food and accomodation, denoted by K, F, G and H respectively.

In [35]:
sql = """
CREATE VIEW business_counts AS 
SELECT 
    s.sa2_code21,
    COUNT(*)::float / p.total_people * 1000 AS business_per_1000
FROM businesses b
JOIN sa2_regions s ON b.sa2_code = s.sa2_code21
JOIN population p ON p.sa2_code = s.sa2_code21
WHERE total_people >= 100 AND s.sa4_code21 = 116 AND 
    b.industry_code IN ('K', 'F', 'G', 'H') AND p.total_people > 0
GROUP BY s.sa2_code21, p.total_people;
"""

query(conn, sql)


Error encountered: 
This result object does not return rows. It has been closed automatically.


""


Number of public transport stops. 

In [36]:
sql = """
CREATE VIEW stop_counts AS
SELECT 
    s.sa2_code21,
    COUNT(*) AS stop_count
FROM sa2_regions s
LEFT JOIN  stops t ON ST_Contains(s.geom, t.geom)
WHERE s.sa4_code21 = 116
GROUP BY s.sa2_code21;
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


School Catchments areas per 1000 ’young people’. 

In [37]:
sql = """
CREATE VIEW school_coverage AS 
SELECT 
s.sa2_code21,
    COUNT(use_id)::float / p.young_people * 1000 AS school_per_1000_youth
FROM primary_sch ps RIGHT JOIN sec_sch ss USING (use_id)
JOIN f_sch fs USING (use_id)
JOIN sa2_regions s ON ST_Intersects(fs.geom, s.geom)
JOIN population p ON p.sa2_code = s.sa2_code21
WHERE p.young_people > 0 and s.sa4_code21 = 116 AND p.total_people >= 100
GROUP BY s.sa2_code21, p.young_people;
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


Number of places of interest, in selected groups. Here our POI groups we choose 1 to 5 as they fit the definition of resourcefulness when we judge a urbanised area in Sydney.

In [38]:
sql = """
CREATE VIEW poi_counts AS
SELECT 
    s.sa2_code21,
    COUNT(*)::float AS poi_count
FROM blacktown_pois p
JOIN sa2_regions s USING (sa2_code21)
WHERE p.poigroup IN (1, 2, 3, 4, 5)
GROUP BY s.sa2_code21;
;
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


Now we combine all the components for the function

In [39]:
sql = """
CREATE VIEW combined_scores AS 
SELECT 
    b.sa2_code21,
    business_per_1000,
    stop_count,
    school_per_1000_youth,
    poi_count
FROM business_counts b
LEFT JOIN stop_counts s ON b.sa2_code21 = s.sa2_code21
LEFT JOIN school_coverage sc ON b.sa2_code21 = sc.sa2_code21
LEFT JOIN poi_counts p ON b.sa2_code21 = p.sa2_code21;
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


Calculate the z-score for each sa2 area and finally the sigmoid score. 

In [42]:
sql = """
SELECT
(business_per_1000 - AVG(business_per_1000) OVER ()) / NULLIF(STDDEV(business_per_1000) OVER (), 0) AS z_business,

(stop_count - AVG(stop_count) OVER ()) / NULLIF(STDDEV(stop_count) OVER (), 0) AS z_stops,

CASE 
    WHEN school_per_1000_youth IS NOT NULL THEN 
        (school_per_1000_youth - AVG(school_per_1000_youth) OVER ()) / NULLIF(STDDEV(school_per_1000_youth) OVER (), 0)
    ELSE 0
END AS z_schools,

(poi_count - AVG(poi_count) OVER ()) / NULLIF(STDDEV(poi_count) OVER (), 0) AS z_poi,


    1 / (1 + EXP(
        -(
            COALESCE(
                (business_per_1000 - AVG(business_per_1000) OVER ()) 
                / NULLIF(STDDEV(business_per_1000) OVER (), 0), 0)
            +
            COALESCE(
                (stop_count - AVG(stop_count) OVER ()) 
                / NULLIF(STDDEV(stop_count) OVER (), 0), 0)
            +
            COALESCE(
                (school_per_1000_youth - AVG(school_per_1000_youth) OVER ()) 
                / NULLIF(STDDEV(school_per_1000_youth) OVER (), 0), 0)
            +
            COALESCE(
                (poi_count - AVG(poi_count) OVER ()) 
                / NULLIF(STDDEV(poi_count) OVER (), 0), 0)
        )
    )) AS score


FROM combined_scores;
"""
query(conn, sql)

Error encountered: 
This Connection is closed


""


In [41]:
conn.close()
db.dispose()